In [22]:
import torch
from torchvision.models import wide_resnet50_2, resnet18
import cv2
import matplotlib.pyplot as plt

from padim_utils import AnomalyDetector,extractEmbeddingVectorsBatched, extractEmbeddingVectors, getParameters, getSeedIndices
from torch.utils.data import DataLoader
import datasets.mvtec as mvtec


## Set device

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

## Load data

In [20]:
class_name = 'up'
data_path = '../data/aligned_pucks/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
dataset.__len__()

x = next(iter(dataloader))[0][0].unsqueeze(0)


## Init model

In [4]:
# model = resnet18(pretrained=True, progress=True)
# model = wide_resnet50_2(pretrained=True, progress=True)
model = resnet18(pretrained=True)

model.to(device)
model.eval()
pass

In [24]:
embedding_vectors = extractEmbeddingVectorsBatched(x, model, device)

In [25]:
embedding_vectors

tensor([[[[0.4741, 0.4741, 0.4741,  ..., 0.5881, 0.5881, 0.5881],
          [0.4741, 0.4741, 0.4741,  ..., 0.5881, 0.5881, 0.5881],
          [0.4741, 0.4741, 0.4741,  ..., 0.5881, 0.5881, 0.5881],
          ...,
          [0.5465, 0.5465, 0.5465,  ..., 0.4845, 0.4845, 0.4845],
          [0.5465, 0.5465, 0.5465,  ..., 0.4845, 0.4845, 0.4845],
          [0.5465, 0.5465, 0.5465,  ..., 0.4845, 0.4845, 0.4845]],

         [[0.0000, 0.1039, 1.2334,  ..., 0.0000, 0.7350, 1.1214],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2409, 0.9744],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2060, 0.7433],
          ...,
          [0.7956, 1.7487, 1.1738,  ..., 0.0000, 0.0000, 0.0000],
          [0.7863, 1.2513, 1.3964,  ..., 0.0000, 0.0000, 0.0000],
          [0.2038, 0.0000, 0.4718,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.7073, 0.7073, 0.5865,  ..., 0.7377, 1.0410, 1.0410],
          [0.7073, 0.7073, 0.5865,  ..., 0.7377, 1.0410, 1.0410],
          [0.6983, 0.6983, 0.9845,  ..., 0

## Extract embedding vectors

In [13]:
embedding_vectors = extractEmbeddingVectors(dataloader, model, device)

| feature extraction | train |: 100%|██████████| 4/4 [00:02<00:00,  1.75it/s]


In [14]:
embedding_vectors.shape

torch.Size([100, 448, 56, 56])

In [18]:
embedding_vectors[10][100]

tensor([[0.0000, 0.0000, 0.2103,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2103,  ..., 0.0000, 0.0000, 0.0000],
        [0.2382, 0.2382, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.4931,  ..., 0.0000, 0.0000, 0.0000],
        [0.0051, 0.0051, 0.1543,  ..., 0.0000, 0.0000, 0.0000],
        [0.0051, 0.0051, 0.1543,  ..., 0.0000, 0.0000, 0.0000]])

In [8]:
# embedding_vectors[:32][0]

In [9]:
idx = getSeedIndices(100, 448, device)  
# idx = getIndices(550, 1792, device)  
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)


In [10]:
embedding_vectors[:32][0]

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.4342, 0.4342, 0.4342,  ..., 0.2712, 0.2712, 0.2712],
         [0.4342, 0.4342, 0.4342,  ..., 0.2712, 0.2712, 0.2712],
         [0.4342, 0.4342, 0.4342,  ..., 0.2712, 0.2712, 0.2712]],

        [[0.0000, 0.1039, 1.2334,  ..., 0.0000, 0.7350, 1.1214],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2409, 0.9744],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2060, 0.7433],
         ...,
         [0.7956, 1.7487, 1.1738,  ..., 0.0000, 0.0000, 0.0000],
         [0.7863, 1.2513, 1.3964,  ..., 0.0000, 0.0000, 0.0000],
         [0.2038, 0.0000, 0.4718,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.3503, 0.3503, 0.3503,  ..., 0.9721, 0.9721, 0.9721],
         [0.3503, 0.3503, 0.3503,  ..., 0.9721, 0.9721, 0.9721],
         [0.3503, 0.3503, 0.3503,  ..., 0.9721, 0.9721, 0.

In [8]:
embedding_vectors.shape

torch.Size([100, 100, 56, 56])

## Calculate mean vectors and covariance matrices

In [9]:
mean, cov = getParameters(embedding_vectors)

In [10]:
mean.shape

(100, 3136)

In [11]:
mean[0]

array([2.3068463e-04, 2.3068463e-04, 2.3068463e-04, ..., 3.0416000e-01,
       3.0416000e-01, 3.0416000e-01], dtype=float32)

## Save values

In [ ]:
# PATH = './batch_example.pkl'
# saveFeatures((mean, cov), PATH)